![SAP Ariba logo](./images/sap-ariba-small.jpeg)  

# Ariba Network Invoice Header Data Extraction API overview

## Overview

API for extracting supplier invoices from the Ariba Network.

**API Documentation**
https://help.sap.com/viewer/9e361d7c0ccc40f5840012785f249811/cloud/en-US/e3f2b8c8a878417bb3ff40db62050f8d.html

## Importing required packages

In [113]:
import json
import requests
import pandas as pd
from pandas.io.json import json_normalize
from IPython.display import display

## Function retrieve authorization token

In [114]:
def get_access_token(base64_auth):
    payload = {}
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Basic {base64_auth}"
    }
    
    response = requests.request("POST", OAUTH_API_URL, headers=headers, data=payload)
    
    return response.json()["access_token"]

## API details

In [115]:
# authentication server
OAUTH_API_URL = "https://api.ariba.com/v2/oauth/token?grant_type=client_credentials"

ANID = "ANXXXXXXXXXX-T" # replace by your own ANID

# API credentials

INV_EXTRACTOR_BASEURL = "https://openapi.ariba.com/api/invoice-extraction/v1/prod/invoices"
INV_EXTRACTOR_APIKEY = "XXXXXX" # replace by your own Key
INV_EXTRACTOR_BASE64 = "XXXXXXX" # replace by your own BASE64 secret


In [116]:
INV_EXTRACTOR_TOKEN = get_access_token(INV_EXTRACTOR_BASE64)
INV_EXTRACTOR_TOKEN

'7bf054b8-74c9-4d0e-ac9c-a10c3639d57a'

## Main Program

In [117]:
api_headers = {
        "Authorization": f"Bearer {INV_EXTRACTOR_TOKEN}",
        "X-ARIBA-NETWORK-ID": ANID,
        "apiKey": INV_EXTRACTOR_APIKEY,
        "accept": "application/json",
    }

#
api_response = requests.get(
    INV_EXTRACTOR_BASEURL,
    headers=api_headers,
)


data = json.loads(api_response.text)["content"]

#flattening json
list_of_invoices = pd.json_normalize(data, max_level=1)

#transforming into DataFrame
invoices = pd.DataFrame(list_of_invoices)

#displaying invoices as a nice table html
display(invoices)

#convert to CSV (optional)
invoices.to_csv(r"invoice_test.csv")



,documentNumber,supplierName,supplierANID,buyerName,buyerANID,createdDate,documentDate,routingStatus,documentStatus,payloadId,...,externalOrderNumber,numberOfItems,purpose,signFromCountry,signToCountry,type,tax.amount,tax.currencyCode,totalAmount.amount,totalAmount.currencyCode
0,testCM,Ariba Beta Supplier,AN01000111002-T,Hawkins - TEST,AN01426057983-T,2021-03-05T08:55:28-08:00,2021-03-05T08:55:13-08:00,Acknowledged,Sent,1614963313190-2878148584269456038@10.162.97.184,...,8100000043,1,lineLevelCreditMemo,US,US,TypeSinglePO,-700,USD,-10700,USD
1,ECCHIN00000024,GLOBAL SUPPLIER,AN01013579508-T,Hawkins - TEST,AN01426057983-T,2021-02-22T08:44:34-08:00,2021-02-22T08:42:28-08:00,Acknowledged,Rejected,z210222164229222654979b3@fieldglass.com,...,8200000025,1,standard,None,None,TypeSinglePO,0,USD,1940,USD
2,InvoiceFG22,GLOBAL SUPPLIER,AN01013579508-T,Hawkins - TEST,AN01426057983-T,2021-02-22T08:38:59-08:00,2021-02-18T02:30:00-08:00,Acknowledged,Approved,1614011932009.931567489.000000441@NnCFEoe+rte+...,...,None,1,standard,None,None,TypeExternalPO,0,USD,1,USD
